# Approach 2.5 / 3.5

Each of the approaches tested so far, despite being used previously in the literature (at least to the extent that I understand them) have failed to out perform just randomly selecting samples. I suspect this could be due to one and/or another issue with the current acquisition strategy. Each of these approaches use some method to find the variance of model predictions to find the most uncertain sequences for the next batch. However, those uncertain sequences might somehow correspond to noisy measurements (measurements with lower read counts, especially if they tend to be the sequences with more mutations), and they might not be sufficiently diverse.

Given these hypotheses for what's going wrong, I may be able to solve them by ensuring the next sample is uncertain as well as diverse. I'd be interested in testing when its just diverse and when its uncertain and diverse compared to the random baseline.

But how do we ensure diversity? Well, since we are conveniently using a language model, we should be able to just use the language model embeddings. So, all I need to do is figure out how to get the embeddings, measure their distance, and integrate some amount of distance optimization into my acquistion function.

Note: Below, I'm going to extract the embeddings from the last hidden state of the CLS token, since that's the standard model I've been working with setting up the structure of active learning. However, other fine-tuning approaches should make use of different embeddings. For example, if you're using only the mutant residues as the final state input to your output layer, you should probably use those embeddings for diversity. Another decision point is whether to use the embeddings of your fine-tuned model or a fresh ESM2 model. I'm going to work with just the fine-tuned model here.

Here's the algorithm I'll employ:
1. Train ensemble on random initial set.
2. Get variances from model predictions.
3. Filter samples with top k% of variances.
4. Retrieve embeddings for those samples.
5. For each sample in the unlabeled pool, calculate its distance from each of the samples in the current batch.
6. Find the minimum distance from those for each candidate.
7. Add candidate c whose minimum distance from the current batch is the greatest and remove it from the unlabeled pool.
8. Calculate the distance of just-added candidate c from each of the remaining unlabeled pool. Add these distances to the distances list for each sample.
9. Repeat 6-8 until n samples are added to the labeled samples.

Actually, now that I spell out the algorithm, I'm realizing that I should also test just a random choice from the top k% of variances to make sure that the embeddings even add anything.

In [1]:
from scripts.data_utils import train_val_test_split
from scripts.config import (
    DATA_PATH, 
    SEQUENCE_COL, 
    SCORE_COL, 
    TOK_MODEL, 
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED,
)

training_pool, val_dataloader, test_dataloader = train_val_test_split(
    DATA_PATH,
    SEQUENCE_COL,
    SCORE_COL,
    TOK_MODEL,
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED
)

In [2]:
import torch
from torch.utils.data import Subset, DataLoader
import numpy as np

from scripts.training import initialize_and_train_new_model
from scripts.acquisition import get_pool_predictions

def get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size):
    bootstrap_indices = np.random.choice(labeled_indices, size=int(0.9*len(labeled_indices)),replace=True)
    bootstrap_subset = Subset(pool_dataset, bootstrap_indices)
    bootstrap_dataloader = DataLoader(bootstrap_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    return bootstrap_dataloader

def train_ensemble(
        n_models, 
        model_name, 
        approach,
        learning_rate,
        weight_decay,
        epochs,
        labeled_indices,
        train_dataloader_batch_size,
        pool_dataset, 
        pool_dataloader, 
        val_dataloader,
        patience
        ):
    
    # define list to store predictions as each model is trained then evaluated
    ensemble_predictions = []
    
    for i in range(n_models):
        print(f"\nTraining Model {i+1}...")
        # set a changing manual seed
        torch.manual_seed(i)
        torch.cuda.manual_seed(i)

        # get bootstrap sample from labeled dataset
        bootstrap_dataloader = get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size)

        # initialize and train a new model
        model = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, bootstrap_dataloader, val_dataloader, patience)
        
        # get model predictions on pool dataloader, append to ensemble predictions list
        pool_preds = get_pool_predictions(model, pool_dataloader)
        ensemble_predictions.append(pool_preds)

    # stack ensemble predictions to create tensor of shape (n_models, n_unlabeled_samples)
    ensemble_predictions = torch.stack(ensemble_predictions, dim=0)
    print("Ensemble training complete, submitting predictions for next cycle.")
    # return list of ensemble predictions
    return ensemble_predictions

### Random choice from top % variances  

In [9]:
import numpy as np
from torch.utils.data import Subset, DataLoader

# acquire new batch, randomly if no scores given, top "batch_size_to_acquire" if given
def acquire_new_batch(
    dataset, 
    train_dataloader_batch_size, 
    pool_dataloader_batch_size, 
    initial_batch_size,
    top_score_fraction, 
    batch_size_to_acquire, 
    labeled_indices, 
    unlabeled_indices, 
    acquisition_scores=None
    ):

    # if initial batch, when there are no acquisition scores, select randomly
    if acquisition_scores is None:
        initial_batch_size = min(initial_batch_size, len(unlabeled_indices))
        indices_to_acquire = np.random.choice(unlabeled_indices, size=initial_batch_size, replace=False)
    
    # else select based on top acquisition scores
    else:
        # make sure we don't overshoot samples to acquire if on the final batch
        batch_size_to_acquire = min(batch_size_to_acquire, len(acquisition_scores))
        # detmine the number of top scorers to select from
        num_top_scorers = int(top_score_fraction * len(unlabeled_indices))
        # get the indicies of the top acquisition scores (num of samples)
        top_indices = acquisition_scores.topk(num_top_scorers).indices
        # choose a random set of indices from these top scorers
        top_k_indices = np.random.choice(top_indices.cpu().numpy(), size=batch_size_to_acquire, replace=False)
        # use these to find the indices that map back to the original dataset
        indices_to_acquire = unlabeled_indices[top_k_indices]
    
    # update the indices lists
    labeled_indices = np.concatenate([labeled_indices, indices_to_acquire])
    unlabeled_indices = np.setdiff1d(unlabeled_indices, indices_to_acquire, assume_unique=True)
    
    # create new subsets and dataloaders
    train_subset = Subset(dataset, labeled_indices.tolist())
    pool_subset = Subset(dataset, unlabeled_indices.tolist())
    train_dataloader = DataLoader(train_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    pool_dataloader = DataLoader(pool_subset, batch_size=pool_dataloader_batch_size, shuffle=False)
    
    return train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices

In [10]:
from pathlib import Path
import pandas as pd

from scripts.acquisition import get_variances
from scripts.training import initialize_and_train_new_model, test_model
from scripts.campaigns import run_standard_finetuning


def get_learning_curves(
        n_samples,
        initial_n_samples,
        top_score_fraction,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            scores = get_variances(ensemble_predictions, f"{results_dir}/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=False)
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, _ = run_standard_finetuning(len(labeled_indices), approach, model_name, train_dataloader_batch_size, learning_rate, weight_decay, epochs, training_pool, val_dataloader, patience)
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(n_models, model_name, approach, learning_rate, weight_decay, epochs, labeled_indices, train_dataloader_batch_size, training_pool, pool_dataloader, val_dataloader, patience)
    
        current_cycle += 1
    return all_results_df

In [11]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
    TOP_SCORE_FRACTION,
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    top_score_fraction=TOP_SCORE_FRACTION,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]: 100%|██████████| 50/50 [00:35<00:00,  1.42it/s]


Train Loss: 0.0336 | Val Loss: 0.1788 | SpearmanR: 0.4060


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 127.40it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]: 100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


Train Loss: 0.0382 | Val Loss: 0.1242 | SpearmanR: 0.6988


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 118.27it/s]

Progress for experiment 15 appended to results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.232308,0.276791,0.255876,0.232170
1,n_samples,0,16,standard,0.208022,0.108390,0.151105,0.207960
2,n_samples,0,16,active,0.225947,0.288939,0.327616,0.226419
3,n_samples,0,16,standard,0.223339,-0.140400,-0.167712,0.223238
4,n_samples,1,32,active,0.184960,0.453173,0.476212,0.185616
5,n_samples,1,32,standard,0.213183,0.243198,0.218171,0.213262
6,n_samples,2,48,active,0.164185,0.492976,0.524374,0.164829
7,n_samples,2,48,standard,0.218176,0.484964,0.408690,0.218421
8,n_samples,3,64,active,0.152050,0.509348,0.531544,0.152574
9,n_samples,3,64,standard,0.144240,0.547185,0.551789,0.143950


In [12]:
lc_df = pd.read_csv("results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv")
lc_df.head()

,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.225947,0.288939,0.327616,0.226419
1,n_samples,0,16,standard,0.223339,-0.140400,-0.167712,0.223238
2,n_samples,1,32,active,0.184960,0.453173,0.476212,0.185616
3,n_samples,1,32,standard,0.213183,0.243198,0.218171,0.213262
4,n_samples,2,48,active,0.164185,0.492976,0.524374,0.164829


In [13]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [14]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/04_adding_diversity/random_from_top_fraction/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,variances5,variances6,variances7,variances8,variances9
0,0.017557,0.017802,0.021863,0.012007,0.007614,0.003172,0.006799,0.021065,0.034405,0.087627,0.007492,0.010298,0.001138,0.001611,0.073645
1,0.030268,0.013252,0.017392,0.014582,0.000798,0.013131,0.012300,0.032668,0.025599,0.013012,0.006858,0.012764,0.006510,0.019254,0.046434
2,0.062635,0.009125,0.005722,0.016405,0.020386,0.018690,0.008005,0.180021,0.043308,0.125256,0.021468,0.057188,0.073069,0.013695,0.049970
3,0.006008,0.047597,0.165927,0.025047,0.017256,0.031446,0.033810,0.034345,0.010135,0.010568,0.011698,0.011238,0.002768,0.009420,0.062151
4,0.108935,0.037727,0.017766,0.017103,0.021608,0.025763,0.023030,0.034272,0.086913,0.054073,0.031779,0.078399,0.056023,0.042748,0.117974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023604,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064340,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022443,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
columns_to_plot = ['variances2', 'variances5', 'variances8', 'variances11', 'variances14']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

### With embeddings

In [ ]:
def get_embeddings(dataset, indices, pool_dataloader_batch_size):
    # Use input indices to make a subset and dataloader from the dataset.
    # initialize a list to store embeddings
    # Loop through the dataloader
        # forward pass on the batch
        # get last hidden states
        # get the embeddings of the first token (CLS token)
        # append to embedding list
    # return embeddings list
    return

def get_distance(embed_1, embed_2):
    # Use cosine distance formula to compare a pair of embeddings: 1 - ((A dot B) / (mag(A) * mag(B)))
    # retrun distance
    return

def get_next_sample():
    # if first sample
        # initialize distance array (final size: # unlabeled samples x # labeled samples)
        # get embeddings of all labeled and candidate samples
        # for each candidate sample
            # for each labeled sample
                # find the distance between candidate and labeled sample
                # append to distance array
        # make minimum distance array (len # candidate samples)
        # get index of candidate sample with maximum distance
        # use candidate indices to map this index to the index in the original dataset
        # get embedding of new sample
        # add embedding of new sample to embeddings list
        # for each candidate sample
            # find the distance between the candidate and new sample
            # append to distance array
        # update indices to acquire and candidate indices lists
        # return indices to acquire, candidate indices, labeled embeddings, candidate embeddings, and distance array
    # if not first sample:
        # make minimum distance array (len # candidate samples)
        # get index of candidate sample with maximum distance
        # use candidate indices to map this index to the index in the original dataset
        # get embedding of new sample
        # add embedding of new sample to embeddings list
        # for each candidate sample
            # find the distance between the candidate and new sample
            # append to distance array
        # update indices to acquire and candidate indices lists
        # return indices to acquire, candidate indices, labeled embeddings, candidate embeddings, and distance array
    return

def acquire_new_batch():
    # if initial batch, when there are no variances, select randomly
    # else select based on top variances scores
        # make sure we don't overshoot samples to acquire if on the final batch
        # detmine the number of top scorers to select from
        # get the indices of the top acquisition scores (num of samples)
        # initialize list of indices to acquire
        # for i in n_samples_per_batch:
            # get next sample, updating indices to acquire
    # update the indices lists
    # create new subsets and dataloaders
    # return train dataloader, pool dataloader, labeled indices, unlabeled indices
    return